# 검색 API 사용하기

보안등 SER를 위해 지번 주소로 도로명 구하기

In [ ]:
from config import vworld_key_search #vworld 인증키 저장한 config.py 파일 불러오기
import pandas as pd
import json
import requests

In [ ]:
#가로등과 보안등 합친 데이터
df = pd.read_csv("입력데이터1.csv")

In [ ]:
#지번 주소가 저장되어 있는 데이터
df_addr = pd.read_csv("입력데이터2.csv")

In [ ]:
#보안등 정보만 가져온 dataframe
lamp2 = df[df["LAMP"] == 2]

lamp2

In [ ]:
#지번 주소 값만 가져오기 - type : str
#리스트를 문자열로 합치는 함수 : join
' '.join(str(df_addr[df_addr['NO'] == 0]['address']).split()[1:6])

### 변수 선언

In [ ]:
#요청 URL를 만들기 위한 변수 선언 

url = 'http://api.vworld.kr/req/search?' #서버 주소
params = 'service=search&request=search&version=2.0&crs=epsg:4326&size=10&page=1&query='#파라미터
key = '&type=address&category=PARCEL&format=json&errorformat=json&key='#나머지 파라미터 및 key
my_key = vworld_key_search #config.py 파일로 설정
road_addr = pd.DataFrame(columns = ['NO', 'road'])

### 요청을 위한 함수 선언

In [ ]:
def request(addr):
    page = requests.get(url+params+addr+key+my_key)
    json_data = json.loads(page.text)
    return json_data

In [ ]:
#이 함수는 현재 내가 가지고 있는 데이터에 맞춰서 변경함
#각 데이터에 맞는 refactoring 필요
def extraction(test_data):
    road_addr = pd.DataFrame(columns = ['NO', 'road'])
    none = None
    for i in range(len(test_data)):
        no = 25587 + i
        addr = ' '.join(str(df_addr[df_addr['NO'] == no]['address']).split()[1:6])
        json_data = request(addr)

        if json_data['response']['status'] == 'NOT_FOUND' or json_data['response']['status'] == 'ERROR': #오류일 때
            road_addr = road_addr.append({'NO':none, 'road':none}, ignore_index=True)
            continue

        road = json_data['response']['result']['items'][0]['address']['road']
        
        road_addr = road_addr.append({'NO':no, 'road':road}, ignore_index=True)
    return road_addr

### 실행

In [ ]:
road_addr = extraction(lamp2)

In [ ]:
road_addr

In [ ]:
#저장
road_addr.to_csv("저장할 이름.csv", index=False)